# Population Health Risk Assessment

Patient Risk Stratification for Value-Based Care
Patient data is analysed to identify who is at risk and what factors drive that risk, so healthcare teams can act proactively.
For this project, SAS files  were downloaded from the NHANES website. These files contain information on population demographics and risk factors such as obesity,a high blood pressure , wether patients smoke and their physical activity status. 

In [453]:
import pandas as pd

In [455]:
demo = pd.read_sas("C:\\Users\\Dell\\OneDrive\\Desktop\\Risk Stratification\\DEMO_L.xpt")
bmx = pd.read_sas("C:\\Users\\Dell\\OneDrive\\Desktop\\Risk Stratification\\BMX_L.xpt")
bpx = pd.read_sas("C:\\Users\\Dell\\OneDrive\\Desktop\\Risk Stratification\\BPXO_L.xpt")
smq = pd.read_sas("C:\\Users\\Dell\\OneDrive\\Desktop\\Risk Stratification\\SMQFAM_L.xpt")
paq = pd.read_sas("C:\\Users\\Dell\\OneDrive\\Desktop\\Risk Stratification\\PAQ_L.xpt")

## Data Extraction and cleaning from the Demographics table

Columns:
patient_id, age, gender, race_ethnicity, income_ratio

What we did:
Checked for missing values → all filled
Ensured age and income_ratio are numeric
Encoded gender as 1/2 (or keep as is)
Ready for merge

Purpose:
Provides basic patient information for dashboards and risk stratification


In [459]:
#From the NHANES demographics table, I retained only patient-level attributes relevant to individual risk stratification and excluded survey design variables intended for population-level inference.

In [461]:
# Selected the columns needed
demo_clean = demo[["SEQN", "RIDAGEYR", "RIAGENDR", "RIDRETH3", "INDFMPIR"]].copy()

In [463]:
#Rename columns 
demo_clean = demo_clean.rename(columns={
    "SEQN": "patient_id",
    "RIDAGEYR": "age",
   "RIAGENDR": "gender",
    "RIDRETH3" :"race_ethnicity",
    "INDFMPIR": "income_ratio"
})
    


In [465]:
demo_clean

,patient_id,age,gender,race_ethnicity,income_ratio
0,130378.0,43.0,1.0,6.0,5.00
1,130379.0,66.0,1.0,3.0,5.00
2,130380.0,44.0,2.0,2.0,1.41
3,130381.0,5.0,2.0,7.0,1.53
4,130382.0,2.0,1.0,3.0,3.60
...,...,...,...,...,...
11928,142306.0,9.0,1.0,2.0,2.01
11929,142307.0,49.0,2.0,4.0,NaN
11930,142308.0,50.0,1.0,2.0,1.95
11931,142309.0,40.0,1.0,2.0,3.11


In [467]:
demo_clean

,patient_id,age,gender,race_ethnicity,income_ratio
0,130378.0,43.0,1.0,6.0,5.00
1,130379.0,66.0,1.0,3.0,5.00
2,130380.0,44.0,2.0,2.0,1.41
3,130381.0,5.0,2.0,7.0,1.53
4,130382.0,2.0,1.0,3.0,3.60
...,...,...,...,...,...
11928,142306.0,9.0,1.0,2.0,2.01
11929,142307.0,49.0,2.0,4.0,NaN
11930,142308.0,50.0,1.0,2.0,1.95
11931,142309.0,40.0,1.0,2.0,3.11


In [469]:
# cleaning spelling of field names 
demo_clean = demo_clean.rename(columns={
    "race_ehtnicity": "race_ethnicity",})

In [471]:
demo_clean.isna().sum()

patient_id           0
age                  0
gender               0
race_ethnicity       0
income_ratio      2041
dtype: int64

In [473]:
# missing values handled with median of the values
demo_clean["income_ratio"] = demo_clean["income_ratio"].fillna(demo_clean["income_ratio"].median())

In [475]:
demo_clean

,patient_id,age,gender,race_ethnicity,income_ratio
0,130378.0,43.0,1.0,6.0,5.00
1,130379.0,66.0,1.0,3.0,5.00
2,130380.0,44.0,2.0,2.0,1.41
3,130381.0,5.0,2.0,7.0,1.53
4,130382.0,2.0,1.0,3.0,3.60
...,...,...,...,...,...
11928,142306.0,9.0,1.0,2.0,2.01
11929,142307.0,49.0,2.0,4.0,2.50
11930,142308.0,50.0,1.0,2.0,1.95
11931,142309.0,40.0,1.0,2.0,3.11


In [477]:
#patient_id changed from decimal form to int.
#NHANES identifiers are stored as floating-point values due to SAS transport formatting, so I explicitly cast patient IDs to integers to ensure data integrity and proper relational joins.
demo_clean["patient_id"] = demo_clean["patient_id"].astype(int)
demo_clean["gender"]=demo_clean["gender"].astype(int)
demo_clean["age"]=demo_clean["age"].astype(int)
demo_clean["race_ethnicity"]=demo_clean["race_ethnicity"].astype(int)

# Data Extraction for Obesity Risk 
Columns:
patient_id, sys1, sys2, sys3, dia1, dia2, dia3

What we did:  

Calculated systolic_avg and diastolic_avg using available readings only (skipna=True)
Added BP_missing_flag → 1 if no readings at all, else 0
Created hypertensive_flag → 1 if systolic ≥130 or diastolic ≥80
Filled missing values where needed

Purpose:
Core clinical risk factor for hypertension
Feeds into risk_score


In [480]:
# Cleaning BMX table 
# is this patient obeses or at healthy body weight i.e Obesity risk
# selected the relevant columns
bmx_clean = bmx[["SEQN","BMXBMI"]].copy()

In [482]:
# renaming columns to more meaningful interpretation 
bmx_clean = bmx[["SEQN","BMXBMI"]].copy()

bmx_clean = bmx_clean.rename(columns={
    "SEQN": "patient_id",
    "BMXBMI": "bmi"
})  # <--- must use columns=..., reassign

bmx_clean["patient_id"] = bmx_clean["patient_id"].astype(int)

In [484]:
# fixing dtype
bmx_clean["patient_id"] = bmx_clean["patient_id"].astype(int)


In [486]:
bmx_clean

,patient_id,bmi
0,130378,27.0
1,130379,33.5
2,130380,29.7
3,130381,23.8
4,130382,NaN
...,...,...
8855,142306,15.4
8856,142307,NaN
8857,142308,26.4
8858,142309,25.5


In [488]:
#Obesity as per CDC standards : BMI >= 30 is obese 
bmx_clean["Obese"]=bmx_clean["bmi"] >=30

In [490]:
#BMI missing values - do not assume obese,do not assume healthy
bmx_clean["Obese"]= bmx_clean ["Obese"].fillna (False)

In [492]:
# copy relevant columns
bmx_clean = bmx[['SEQN', 'BMXBMI']].copy()

# rename columns
bmx_clean.rename(columns={'SEQN': 'patient_id', 'BMXBMI': 'BMI'}, inplace=True)



In [494]:
# convert types
bmx_clean['patient_id'] = bmx_clean['patient_id'].astype(int)
bmx_clean['BMI'] = pd.to_numeric(bmx_clean['BMI'], errors='coerce')


In [496]:
# create obesity_flag (BMI ≥ 30)
bmx_clean['obesity_flag'] = (bmx_clean['BMI'] >= 30).astype(int)

# keep only patient_id and obesity_flag
bmx_clean = bmx_clean[['patient_id', 'obesity_flag']]

bmx_clean.head()

#transformed raw  measurements into clinically meaningful obesity risk indicators using CDC BMI thresholds.

,patient_id,obesity_flag
0,130378,0
1,130379,1
2,130380,0
3,130381,0
4,130382,0


## Data extraction and cleaning for blood pressure measurements
Columns:  
patient_id, sys1, sys2, sys3, dia1, dia2, dia3

What we did:  
Calculated systolic_avg and diastolic_avg using available readings only (skipna=True)
Added BP_missing_flag → 1 if no readings at all, else 0
Created hypertensive_flag → 1 if systolic ≥130 or diastolic ≥80
Filled missing values where needed

Purpose:  
Core clinical risk factor for hypertension
Feeds into risk_score

In [499]:
bpx

,SEQN,BPAOARM,BPAOCSZ,BPXOSY1,BPXODI1,BPXOSY2,BPXODI2,BPXOSY3,BPXODI3,BPXOPLS1,BPXOPLS2,BPXOPLS3
0,130378.0,b'R',4.0,135.0,98.0,131.0,96.0,132.0,94.0,82.0,79.0,82.0
1,130379.0,b'R',4.0,121.0,84.0,117.0,76.0,113.0,76.0,72.0,71.0,73.0
2,130380.0,b'R',4.0,111.0,79.0,112.0,80.0,104.0,76.0,84.0,83.0,77.0
3,130386.0,b'R',4.0,110.0,72.0,120.0,74.0,115.0,75.0,59.0,64.0,64.0
4,130387.0,b'R',4.0,143.0,76.0,136.0,74.0,145.0,78.0,80.0,80.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7796,142306.0,b'R',2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7797,142307.0,b'R',4.0,127.0,75.0,132.0,73.0,131.0,72.0,71.0,70.0,67.0
7798,142308.0,b'R',3.0,106.0,65.0,106.0,69.0,112.0,74.0,58.0,61.0,69.0
7799,142309.0,b'R',3.0,127.0,81.0,125.0,82.0,128.0,81.0,80.0,79.0,83.0


In [500]:
# Cleaning BP table - keeping required columns 
bpx_clean = bpx[["SEQN", "BPXOSY1", "BPXOSY2", "BPXOSY3","BPXODI1", "BPXODI2", "BPXODI3"]].copy()

In [503]:
#renaming for better intrepretation
bpx_clean.rename(columns= {
    "SEQN": "patient_id",
    "BPXOSY1": "sys1", "BPXOSY2": "sys2", "BPXOSY3": "sys3",
     "BPXODI1": "dia1", "BPXODI2": "dia2", "BPXODI3": "dia3"
},inplace=True)


In [504]:
# changing relevant datatypes 
bpx_clean["patient_id"] = bpx_clean["patient_id"].astype(int)

In [507]:
bpx_clean

,patient_id,sys1,sys2,sys3,dia1,dia2,dia3
0,130378,135.0,131.0,132.0,98.0,96.0,94.0
1,130379,121.0,117.0,113.0,84.0,76.0,76.0
2,130380,111.0,112.0,104.0,79.0,80.0,76.0
3,130386,110.0,120.0,115.0,72.0,74.0,75.0
4,130387,143.0,136.0,145.0,76.0,74.0,78.0
...,...,...,...,...,...,...,...
7796,142306,NaN,NaN,NaN,NaN,NaN,NaN
7797,142307,127.0,132.0,131.0,75.0,73.0,72.0
7798,142308,106.0,106.0,112.0,65.0,69.0,74.0
7799,142309,127.0,125.0,128.0,81.0,82.0,81.0


In [509]:
bpx_clean.isnull().sum()

patient_id      0
sys1          284
sys2          296
sys3          321
dia1          284
dia2          296
dia3          321
dtype: int64

In [511]:
# Blood pressure readings have alot of missing values so even when i take an averga , it is expected and avergae number of Missing values will be found. 
# A good solution is to not include the Null values when calculating average. to avoid large missing dataset.
#Average available BP readings per patient (row-wise), ignoring missing values, and only treat BP as missing if all readings are missing.


In [513]:
# Row-wise average of available systolic readings
bpx_clean['systolic_avg'] = bpx_clean[['sys1', 'sys2', 'sys3']].mean(axis=1, skipna=True)

# Row-wise average of available diastolic readings
bpx_clean['diastolic_avg'] = bpx_clean[['dia1', 'dia2', 'dia3']].mean(axis=1, skipna=True)

bpx_clean['systolic_avg'] = pd.to_numeric(bpx_clean['systolic_avg'], errors='coerce')
bpx_clean['diastolic_avg'] = pd.to_numeric(bpx_clean['diastolic_avg'], errors='coerce')

In [515]:
bpx_clean['hypertensive_flag'] = (
    (bpx_clean['systolic_avg'] >= 130) | (bpx_clean['diastolic_avg'] >= 80)
).astype(int)


In [517]:
bpx_clean

,patient_id,sys1,sys2,sys3,dia1,dia2,dia3,systolic_avg,diastolic_avg,hypertensive_flag
0,130378,135.0,131.0,132.0,98.0,96.0,94.0,132.666667,96.000000,1
1,130379,121.0,117.0,113.0,84.0,76.0,76.0,117.000000,78.666667,0
2,130380,111.0,112.0,104.0,79.0,80.0,76.0,109.000000,78.333333,0
3,130386,110.0,120.0,115.0,72.0,74.0,75.0,115.000000,73.666667,0
4,130387,143.0,136.0,145.0,76.0,74.0,78.0,141.333333,76.000000,1
...,...,...,...,...,...,...,...,...,...,...
7796,142306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7797,142307,127.0,132.0,131.0,75.0,73.0,72.0,130.000000,73.333333,1
7798,142308,106.0,106.0,112.0,65.0,69.0,74.0,108.000000,69.333333,0
7799,142309,127.0,125.0,128.0,81.0,82.0,81.0,126.666667,81.333333,1


In [519]:
bpx_clean['hypertensive_flag'].value_counts()

hypertensive_flag
0    5297
1    2504
Name: count, dtype: int64

In [521]:
bpx_clean[['patient_id','systolic_avg','diastolic_avg','hypertensive_flag']].head(20)

,patient_id,systolic_avg,diastolic_avg,hypertensive_flag
0,130378,132.666667,96.000000,1
1,130379,117.000000,78.666667,0
2,130380,109.000000,78.333333,0
3,130386,115.000000,73.666667,0
4,130387,141.333333,76.000000,1
5,130388,129.000000,99.666667,1
6,130389,133.000000,79.333333,1
7,130390,111.333333,74.666667,0
8,130391,106.000000,72.666667,0
9,130392,158.333333,71.333333,1


In [523]:
#selecting relevant columns for analysis from this dataframe.
bpx_clean = bpx_clean[["patient_id","systolic_avg","diastolic_avg","hypertensive_flag"]]


In [525]:
bpx_clean

,patient_id,systolic_avg,diastolic_avg,hypertensive_flag
0,130378,132.666667,96.000000,1
1,130379,117.000000,78.666667,0
2,130380,109.000000,78.333333,0
3,130386,115.000000,73.666667,0
4,130387,141.333333,76.000000,1
...,...,...,...,...
7796,142306,NaN,NaN,0
7797,142307,130.000000,73.333333,1
7798,142308,108.000000,69.333333,0
7799,142309,126.666667,81.333333,1


## Data extraction and cleaning for behavioural factor such as smoking status.
Columns:  
patient_id, ever_smoked, current_smoker

What we did:  
Filled missing values with 0 → assumes never smoked / not current smoker
Created smoker_flag for risk score
Optional missing flag for auditing

Purpose:  
Smoking is another key risk factor for patient stratification

In [528]:
smq

,SEQN,SMD460,SMD470
0,130378.0,5.397605e-79,NaN
1,130379.0,5.397605e-79,NaN
2,130380.0,5.397605e-79,NaN
3,130381.0,5.397605e-79,NaN
4,130382.0,5.397605e-79,NaN
...,...,...,...
11928,142306.0,5.397605e-79,NaN
11929,142307.0,2.000000e+00,5.397605e-79
11930,142308.0,5.397605e-79,NaN
11931,142309.0,1.000000e+00,5.397605e-79


In [530]:
smq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11933 entries, 0 to 11932
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SEQN    11933 non-null  float64
 1   SMD460  10624 non-null  float64
 2   SMD470  2604 non-null   float64
dtypes: float64(3)
memory usage: 279.8 KB


In [532]:
smq.describe()

,SEQN,SMD460,SMD470
count,11933.000000,1.062400e+04,2.604000e+03
mean,136344.000000,1.992470e+00,8.586790e-01
std,3444.904716,3.685317e+01,1.957984e+01
min,130378.000000,5.397605e-79,5.397605e-79
25%,133361.000000,5.397605e-79,5.397605e-79
50%,136344.000000,5.397605e-79,5.397605e-79
75%,139327.000000,5.397605e-79,1.000000e+00
max,142310.000000,9.990000e+02,9.990000e+02


In [534]:
# select only relevant columns
smq_clean = smq[["SEQN","SMD460","SMD470"]].copy()

In [536]:
# renaming columns
smq_clean.rename(columns={
    "SEQN": "patient_id",
    "SMD460": "ever_smoked",
    "SMD470": "current_smoker"
}, inplace=True)

In [538]:
# replace invalid NHANES codes with NaN
smq_clean['ever_smoked'] = smq_clean['ever_smoked'].replace(
    [7, 9, 77, 99, 777, 999, 5.397605e-79], pd.NA
)

smq_clean['current_smoker'] = smq_clean['current_smoker'].replace(
    [7, 9, 77, 99, 777, 999, 5.397605e-79], pd.NA
)

# create smoker flag
smq_clean['smoker_flag'] = (
    (smq_clean['ever_smoked'] == 1) &
    (smq_clean['current_smoker'] == 1)
).astype(int)

# keep only what you need
smq_clean = smq_clean[['patient_id', 'smoker_flag']]


In [540]:
smq_clean['smoker_flag'].value_counts()

smoker_flag
0    11327
1      606
Name: count, dtype: int64

In [542]:
#convert patient_id to integer
smq_clean["patient_id"] = smq_clean["patient_id"].astype(int)


In [544]:
smq_clean

,patient_id,smoker_flag
0,130378,0
1,130379,0
2,130380,0
3,130381,0
4,130382,0
...,...,...
11928,142306,0
11929,142307,0
11930,142308,0
11931,142309,0


In [546]:
#Ever smoked or current smoker missing value can be 0 

In [548]:
#I translated raw NHANES smoking questionnaire data into a binary current-smoking risk indicator, consistent with clinical risk stratification used in value-based care.
smq_clean.head(42)

,patient_id,smoker_flag
0,130378,0
1,130379,0
2,130380,0
3,130381,0
4,130382,0
5,130383,0
6,130384,0
7,130385,0
8,130386,0
9,130387,0


## Data extraction and cleaning for physical activity dataframe
Columns:  
patient_id, physically_inactive_flag

What we did:  
Filled missing values with 0 → assumes physically active if missing
Converted to integer (0/1)

Purpose:  
Risk factor for chronic disease / high BP

In [551]:
paq

,SEQN,PAD790Q,PAD790U,PAD800,PAD810Q,PAD810U,PAD820,PAD680
0,130378.0,3.000000e+00,b'W',45.0,3.000000e+00,b'W',45.0,360.0
1,130379.0,4.000000e+00,b'W',45.0,3.000000e+00,b'W',45.0,480.0
2,130380.0,1.000000e+00,b'W',20.0,5.397605e-79,b'',NaN,240.0
3,130384.0,5.397605e-79,b'',NaN,5.397605e-79,b'',NaN,60.0
4,130385.0,1.000000e+00,b'D',90.0,1.000000e+00,b'W',60.0,180.0
...,...,...,...,...,...,...,...,...
8148,142305.0,2.000000e+00,b'W',40.0,5.397605e-79,b'',NaN,480.0
8149,142307.0,3.000000e+00,b'W',15.0,5.397605e-79,b'',NaN,480.0
8150,142308.0,1.000000e+00,b'W',45.0,5.397605e-79,b'',NaN,600.0
8151,142309.0,2.000000e+00,b'D',15.0,5.397605e-79,b'',NaN,240.0


In [553]:
paq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8153 entries, 0 to 8152
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SEQN     8153 non-null   float64
 1   PAD790Q  8135 non-null   float64
 2   PAD790U  8153 non-null   object 
 3   PAD800   6390 non-null   float64
 4   PAD810Q  8139 non-null   float64
 5   PAD810U  8153 non-null   object 
 6   PAD820   3687 non-null   float64
 7   PAD680   8138 non-null   float64
dtypes: float64(6), object(2)
memory usage: 509.7+ KB


In [555]:
paq.describe()

,SEQN,PAD790Q,PAD800,PAD810Q,PAD820,PAD680
count,8153.000000,8.135000e+03,6390.000000,8.139000e+03,3687.000000,8.138000e+03
mean,136378.394579,6.014247e+01,93.437715,4.921084e+01,97.571467,4.469827e+02
std,3440.455897,7.417083e+02,544.415142,6.852722e+02,605.421908,9.174642e+02
min,130378.000000,5.397605e-79,1.000000,5.397605e-79,1.000000,5.397605e-79
25%,133395.000000,1.000000e+00,30.000000,5.397605e-79,30.000000,1.800000e+02
50%,136419.000000,2.000000e+00,60.000000,5.397605e-79,45.000000,3.000000e+02
75%,139346.000000,4.000000e+00,60.000000,2.000000e+00,60.000000,4.800000e+02
max,142310.000000,9.999000e+03,9999.000000,9.999000e+03,9999.000000,9.999000e+03


In [557]:
paq.isna().sum()

SEQN          0
PAD790Q      18
PAD790U       0
PAD800     1763
PAD810Q      14
PAD810U       0
PAD820     4466
PAD680       15
dtype: int64

In [559]:
# select only what you need
paq_clean = paq[["SEQN", "PAD680"]].copy()

In [561]:
# rename columns
paq_clean.rename(columns={
    "SEQN": "patient_id",
    "PAD680": "total_min_per_week"
}, inplace=True)

In [563]:
# fix patient_id type
paq_clean["patient_id"] = paq_clean["patient_id"].astype(int)

In [565]:
# ensure numeric
#I explicitly declare this column to be numeric and I control how bad data is handled.
paq_clean["total_min_per_week"] = pd.to_numeric(
    paq_clean["total_min_per_week"], errors="coerce"
)

In [567]:
# create inactivity flag
paq_clean["physically_inactive_flag"] = (
    paq_clean["total_min_per_week"] < 150
).astype(int)

In [569]:
# treat missing as inactive
paq_clean["physically_inactive_flag"] = paq_clean["physically_inactive_flag"].fillna(1)

In [571]:
# keep final columns only
paq_clean = paq_clean[["patient_id", "physically_inactive_flag"]]

In [573]:
paq_clean

,patient_id,physically_inactive_flag
0,130378,0
1,130379,0
2,130380,0
3,130384,1
4,130385,0
...,...,...
8148,142305,0
8149,142307,0
8150,142308,0
8151,142309,0


#  Merged all dataframes into a single dataframe

Filled any remaining missing flags with 0  
Calculated risk_score = sum of 4 flags (obesity_flag, hypertensive_flag, smoker_flag, physically_inactive_flag)  
Converted risk_score to risk_category (Low / Medium / High)

In [576]:
# Merging the final dataframe

In [578]:
# Merging tables on patient_id
#I merged multiple NHANES datasets into a single patient-level master table, created clinically validated binary risk flags for obesity, hypertension, smoking, and physical inactivity, and made it ready for risk stratification dashboards in Power BI.


In [580]:
# merge all tables on patient_id
master_table = demo_clean.merge(bmx_clean, on="patient_id", how="left") \
                         .merge(bpx_clean[['patient_id','hypertensive_flag']], on="patient_id", how="left") \
                         .merge(smq_clean, on="patient_id", how="left") \
                         .merge(paq_clean, on="patient_id", how="left")


# fill any missing flags with 0 
master_table['obesity_flag'] = master_table['obesity_flag'].fillna(0).astype(int)
master_table['hypertensive_flag'] = master_table['hypertensive_flag'].fillna(0).astype(int)
master_table['smoker_flag'] = master_table['smoker_flag'].fillna(0).astype(int)
master_table['physically_inactive_flag'] = master_table['physically_inactive_flag'].fillna(0).astype(int)




In [583]:
risk_columns = ['obesity_flag', 'hypertensive_flag', 'smoker_flag', 'physically_inactive_flag']
for col in risk_columns:
    master_table[col] = master_table[col].fillna(0).astype(int)


In [585]:
master_table[['obesity_flag','hypertensive_flag','smoker_flag','physically_inactive_flag']].isnull().sum()


obesity_flag                0
hypertensive_flag           0
smoker_flag                 0
physically_inactive_flag    0
dtype: int64

In [587]:
# create risk score
master_table['risk_score'] = (
    master_table['obesity_flag'] +
    master_table['hypertensive_flag'] +
    master_table['smoker_flag'] +
    master_table['physically_inactive_flag']
)

In [590]:
#create risk categories
def risk_category(score):
    if score == 0:
        return 'Low'
    elif score in [1, 2]:
        return 'Medium'
    else:
        return 'High'

master_table['risk_category'] = master_table['risk_score'].apply(risk_category)

# check
master_table[['patient_id','risk_score','risk_category']].head()


,patient_id,risk_score,risk_category
0,130378,1,Medium
1,130379,1,Medium
2,130380,0,Low
3,130381,0,Low
4,130382,0,Low


In [592]:
master_table[risk_columns + ['risk_score']].sum()


obesity_flag                2699
hypertensive_flag           2504
smoker_flag                  606
physically_inactive_flag    1168
risk_score                  6977
dtype: int64

In [594]:
master_table['risk_category'].value_counts()


risk_category
Low       6880
Medium    4837
High       216
Name: count, dtype: int64

In [596]:
master_table.isna().sum()

patient_id                  0
age                         0
gender                      0
race_ethnicity              0
income_ratio                0
obesity_flag                0
hypertensive_flag           0
smoker_flag                 0
physically_inactive_flag    0
risk_score                  0
risk_category               0
dtype: int64

In [598]:
# sanity check 
master_table[master_table["hypertensive_flag"]==1]
master_table[master_table["obesity_flag"]==1]
master_table[master_table["physically_inactive_flag"]==1]

,patient_id,age,gender,race_ethnicity,income_ratio,obesity_flag,hypertensive_flag,smoker_flag,physically_inactive_flag,risk_score,risk_category
6,130384,43,1,1,0.63,0,0,0,1,1,Medium
20,130398,47,1,1,1.67,0,0,0,1,1,Medium
26,130404,73,2,3,4.42,0,1,0,1,2,Medium
30,130408,80,1,4,1.02,0,1,0,1,2,Medium
32,130410,77,2,3,3.62,0,0,0,1,1,Medium
...,...,...,...,...,...,...,...,...,...,...,...
11869,142247,43,1,3,2.50,0,0,0,1,1,Medium
11886,142264,63,2,2,2.16,0,0,0,1,1,Medium
11921,142299,33,1,1,3.28,1,0,0,1,2,Medium
11922,142300,46,2,1,0.68,1,1,0,1,3,High


# Finalised CSV file for continuing workflow

In [601]:
master_table.to_csv("C:\\Users\\Dell\\OneDrive\\Desktop\\Risk Stratification\\master_risk_table.csv", index=False)

In [609]:
master_table

,patient_id,age,gender,race_ethnicity,income_ratio,obesity_flag,hypertensive_flag,smoker_flag,physically_inactive_flag,risk_score,risk_category
0,130378,43,1,6,5.00,0,1,0,0,1,Medium
1,130379,66,1,3,5.00,1,0,0,0,1,Medium
2,130380,44,2,2,1.41,0,0,0,0,0,Low
3,130381,5,2,7,1.53,0,0,0,0,0,Low
4,130382,2,1,3,3.60,0,0,0,0,0,Low
...,...,...,...,...,...,...,...,...,...,...,...
11928,142306,9,1,2,2.01,0,0,0,0,0,Low
11929,142307,49,2,4,2.50,0,1,0,0,1,Medium
11930,142308,50,1,2,1.95,0,0,0,0,0,Low
11931,142309,40,1,2,3.11,0,1,0,0,1,Medium


## Preparing File for importing in SQL

In [604]:
import pandas as pd
import sqlite3

# Connect to your SQLite DB
conn = sqlite3.connect('patient_risk_master.db')

# Read table into DataFrame
df = pd.read_sql_query("SELECT * FROM patient_risk_master", conn)

# Export to CSV
df.to_csv("C:/Users/Dell/OneDrive/Desktop/Risk Stratification/patient_risk_master.csv", index=False)

conn.close()



In [607]:
master_table

,patient_id,age,gender,race_ethnicity,income_ratio,obesity_flag,hypertensive_flag,smoker_flag,physically_inactive_flag,risk_score,risk_category
0,130378,43,1,6,5.00,0,1,0,0,1,Medium
1,130379,66,1,3,5.00,1,0,0,0,1,Medium
2,130380,44,2,2,1.41,0,0,0,0,0,Low
3,130381,5,2,7,1.53,0,0,0,0,0,Low
4,130382,2,1,3,3.60,0,0,0,0,0,Low
...,...,...,...,...,...,...,...,...,...,...,...
11928,142306,9,1,2,2.01,0,0,0,0,0,Low
11929,142307,49,2,4,2.50,0,1,0,0,1,Medium
11930,142308,50,1,2,1.95,0,0,0,0,0,Low
11931,142309,40,1,2,3.11,0,1,0,0,1,Medium


In [443]:
master_table[master_table["smoker_flag"]==1]

,patient_id,age,gender,race_ethnicity,income_ratio,obesity_flag,systolic_avg,diastolic_avg,hypertensive_flag,BP_missing_flag,smoker_flag,physically_inactive_flag,risk_score,risk_category
40,130418,53,1,3,0.20,1,86.333333,57.333333,0,0,1,1,3,High
41,130419,18,2,6,2.64,1,116.333333,78.666667,0,0,1,0,2,Medium
48,130426,67,1,1,0.88,0,166.333333,88.000000,1,0,1,0,2,Medium
70,130448,13,2,1,1.81,0,NaN,NaN,0,1,1,0,1,Medium
113,130491,69,2,7,0.79,1,89.666667,60.666667,0,0,1,0,2,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11801,142179,24,2,7,0.21,0,97.666667,55.333333,0,0,1,0,1,Medium
11825,142203,59,1,2,2.02,0,191.666667,115.666667,1,0,1,0,2,Medium
11846,142224,37,1,7,5.00,0,NaN,NaN,0,1,1,1,2,Medium
11863,142241,51,1,3,4.22,1,120.666667,79.000000,0,0,1,0,2,Medium


In [448]:
master_table.to_csv("C:\\Users\\Dell\\OneDrive\\Desktop\\Risk Stratification\\master_risk_table.csv", index=False)